In [19]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from docx import Document
from docx.shared import Pt  # For font size
from docx.oxml.ns import qn  # For language customization
from docx.oxml import parse_xml  # For custom XML styling
from docx.shared import RGBColor  # For font color
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement
import random
import string


In [20]:
chrome_options =  webdriver.ChromeOptions()
title_search_news_list=[]
chrome_options.add_argument("--disable-gpu")  
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2,}
)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_argument("--mute-audio")
chrome_options.add_argument("--autoplay-policy=no-user-gesture-required")# Abort the request if it's a video file

doc = Document()
sentence=''

In [21]:
service = Service("C:/Users/mehdi/Documents/webdrivers/chromedriver.exe")
driver = webdriver.Chrome(service=service,options=chrome_options)  

In [22]:
driver.get('https://www.google.com/advanced_search')

In [23]:
current_sentence=''
news_websites_url=['https://akharinkhabar.ir',#0
                   'https://nournews.ir',#1
                   'https://isna.ir',#2
                   'https://farsnews.ir',#3
                   'https://www.irna.ir',#4
                   'https://www.iribnews.ir',#5
                   'https://www.mehrnews.com',#6
                   'https://www.tasnimnews.com'#7
                   ]
key_word = ["لبنان", "غزه", "ایران"]
result_map={}
custom_key=0

In [24]:
public_url=news_websites_url[1]#choose webisite

In [25]:
def compare(key,text):
    print('----------------------------')
    print(f'sentence{sentence}')
    total_count = sum(text.lower().split().count(word.lower()) for word in key_word)
    print(f'custom_key{key}')
    # Create the result list
    result = [total_count, text]
    print(f'compare customkey{key}')
    # Add the result to a map with a custom key
     # Define your custom key
    result_map[''.join(random.choices(string.ascii_letters + string.digits, k=10))]=result
    
    # Output
    print("Result:", result)
    print("Map:", result_map)

In [26]:
def akharinKhabarnews():
    print('akharin')
    sentence=driver.find_element(By.CLASS_NAME, "content_content__sfzd5").text
    return sentence
    print(sentence)
    print('akharin done!')
    

In [27]:
def tasnim():
    sentence=str(driver.find_element(By.CLASS_NAME, "story"))
    return sentence
    print('tasni done!')

In [28]:
def iribnews():
    container= str(driver.find_element(By.CLASS_NAME, "div.news_center_body"))
    dive= str(driver.find_element(By.CLASS_NAME, "div.body.body_media_content_show"))
    sentence=f"{container} \n {dive}"
    return sentence
    print('iribnews done!')

In [29]:
def noornews():
    sentence= driver.find_element(By.XPATH, '//*[@id="div_balance_right"]/section/div').text
    return sentence
    print('noor done!')

In [30]:
def irna():
    elements = driver.find_elements(By.CSS_SELECTOR, ".item-text p")
    sentence=" ".join([element.text for element in elements])
    return sentence
    print('irna done!')

In [31]:
def fars():
    news_text_elements=driver.find_element(By.CSS_SELECTOR, ".n-nqnd1g span.auto-dir-block")
    sentence=" ".join([element.text for element in news_text_elements])
    return sentence
    print('fars done!')

In [32]:
def mehrnews():
    sentence=driver.find_element(By.CSS_SELECTOR, 'div.item-body div.item-text').text
    return sentence
    print('mehr done!')
    

In [33]:
def isna():
    summary = driver.find_element(By.CSS_SELECTOR, ".summary").text
    article_body = driver.find_element(By.CSS_SELECTOR, ".item-text").text
    sentence=f"{summary}\n\n{article_body}"
    return sentence
    print(' isna done!')

In [34]:
# Fill the 'all these words' field
all_words = driver.find_element(By.ID, 'xX4UFf')
all_words.send_keys('سوریه لبنان')

# Fill the 'this exact word or phrase' field
exact_phrase = driver.find_element(By.ID, 'CwYCWc')
exact_phrase.send_keys('صلح')

# Fill the 'any of these words' field
any_words = driver.find_element(By.ID, 'mSoczb')
any_words.send_keys(' تهران')

# Fill the 'none of these words' field
none_words = driver.find_element(By.ID, 't2dX1c')
none_words.send_keys('نتانیاهو')

# Fill the 'numbers ranging from' field (start and end)
# range_start = driver.find_element(By.ID, 'LK5akc')
# range_start.send_keys('10')

# range_end = driver.find_element(By.NAME, 'as_nhi')  # Note: 'as_nhi' doesn't have an ID but has a name attribute
# range_end.send_keys('35 kg')
driver.find_element(By.ID, 'NqEkZd').send_keys(public_url.replace("https://", "").replace("http://", ""))  # Fill 'Website or domain'
advance_button=driver.find_element(By.XPATH,'//*[@id="s1zaZb"]/div[5]/div[8]/div[2]/input[2]')
advance_button.click()
custom_key=0

In [35]:
data = []
news_current_urls = []  # Variable to store all URLs
elements = driver.find_elements(By.CLASS_NAME, "LC20lb")  # Locate elements with the specific class
for element in elements:
    title = element.text  # Get the title text
    try:
        parent = element.find_element(By.XPATH, "..")  # Move to the parent element (usually <a>)
        url = parent.get_attribute("href")  # Get the link
        if title and url:
            data.append({"title": title, "url": url})
            news_current_urls.append(url)  # Save the URL in the variable
    except Exception as e:
        print(f"Error processing element: {e}")

# Print Titles and URLs
print("Extracted Titles and URLs:")
for item in data:
    print(f"Title: {item['title']}")
    title_search_news_list.append(item['title'])
    
    print(f"URL: {item['url']}")
    print("-" * 50)

# Print URLs stored in `news_current_urls`
print("All URLs:")
print(news_current_urls)

Extracted Titles and URLs:
Title: ظریف: ایران و روسیه به دنبال صلح در منطقه هستند
URL: https://nournews.ir/fa/news/38795/%D8%B8%D8%B1%DB%8C%D9%81-%D8%A7%DB%8C%D8%B1%D8%A7%D9%86-%D9%88-%D8%B1%D9%88%D8%B3%DB%8C%D9%87-%D8%A8%D9%87-%D8%AF%D9%86%D8%A8%D8%A7%D9%84-%D8%B5%D9%84%D8%AD-%D8%AF%D8%B1-%D9%85%D9%86%D8%B7%D9%82%D9%87-%D9%87%D8%B3%D8%AA%D9%86%D8%AF
--------------------------------------------------
Title: آمریکا 11 فرد و نهاد سوریه را تحریم کرد
URL: https://nournews.ir/fa/news/168584/%D8%A2%D9%85%D8%B1%DB%8C%DA%A9%D8%A7-11-%D9%81%D8%B1%D8%AF-%D9%88-%D9%86%D9%87%D8%A7%D8%AF-%D8%B3%D9%88%D8%B1%DB%8C%D9%87-%D8%B1%D8%A7-%D8%AA%D8%AD%D8%B1%DB%8C%D9%85-%DA%A9%D8%B1%D8%AF
--------------------------------------------------
Title: چندرسانه‌ای: چرا ائتلاف ایران، سوریه و روسیه، غرب را به هراس ...
URL: https://nournews.ir/fa/news/59206/%DA%86%D9%86%D8%AF%D8%B1%D8%B3%D8%A7%D9%86%D9%87%E2%80%8C%D8%A7%DB%8C-%DA%86%D8%B1%D8%A7-%D8%A7%D8%A6%D8%AA%D9%84%D8%A7%D9%81-%D8%A7%DB%8C%D8%B1%D8%A7%D9%86%D8%8C

In [36]:
for index, url in enumerate(news_current_urls):
    print(f"Loading website {index + 1}/{len(news_current_urls)}: {url}")
    driver.get(url)  # Load the URL
    custom_key=custom_key +1
    
        # Wait until the page has fully loaded (specific condition can be customized)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))  # Wait for the body tag to load
    )
        
    print(f'public:{public_url}')
    try:
        match public_url:
            case   'https://nournews.ir':                    compare(custom_key,noornews())
            case   'https://isna.ir': compare(custom_key,isna())
            case   'https://farsnews.ir': compare(custom_key,fars())
            case   'https://www.irna.ir': compare(custom_key,irna())
            case   'https://www.iribnews.ir': compare(custom_key,iribnews())
            case   'https://www.mehrnews.com': compare(custom_key,mehrnews())
            case   'https://www.tasnimnews.com': compare(custom_key,tasnim()) 
            case   'https://akharinkhabar.ir':compare(custom_key,akharinKhabarnews())
    except Exception as e:
        print(f"Error processing element: {e}")
        continue
        
    print(f"Website {url} fully loaded.")
    

# Close the browser
print("All websites processed.")

Loading website 1/9: https://nournews.ir/fa/news/38795/%D8%B8%D8%B1%DB%8C%D9%81-%D8%A7%DB%8C%D8%B1%D8%A7%D9%86-%D9%88-%D8%B1%D9%88%D8%B3%DB%8C%D9%87-%D8%A8%D9%87-%D8%AF%D9%86%D8%A8%D8%A7%D9%84-%D8%B5%D9%84%D8%AD-%D8%AF%D8%B1-%D9%85%D9%86%D8%B7%D9%82%D9%87-%D9%87%D8%B3%D8%AA%D9%86%D8%AF


public:https://nournews.ir
----------------------------
sentence
custom_key1
compare customkey1
Result: [8, 'ظریف: ایران و روسیه به دنبال صلح در منطقه هستند\nوزیر امور خارجه جمهوری اسلامی ایران اعلام کرد: ایران و روسیه بر خلاف آمریکا به دنبال صلح و ثبات در منطقه هستند و طرح های ارایه شده از سوی دو کشور نشانگر این واقعیت است.\n «محمد جواد ظریف» در دیدار با سرگئی لاوروف افزود: روابط دوجانبه ایران و روسیه امروز شاید در بهترین شرایط تاریخی خود قرار دارد.\nوی اظهار داشت: برای دو کشور مهم در منطقه این روابط راهبردی است که همواره با قدرت ادامه پیدا خواهد کرد چرا که این اراده مسئولان جمهوری اسلامی است که به روابط با روسیه به عنوان یک همسایه بزرگ، مهم و یک قدرت بزرگ بین\u200cالمللی به صورت راهبردی نگاه کنند.\nوی ادامه داد: برخلاف دیگران که در منطقه ما به جنگ افروزی دامن می\u200cزنند ایران و روسیه پیشنهادهای مهمی که بسیار هم شبیه به هم هستند برای صلح در منطقه ارایه کرده\u200cاند. \nظریف ارایه طرح گفت\u200cوگوی روسیه در خلیج فارس و طرح صلح هرمز را در راستای چنین سیاسی عنوان کرد.\nوزیر امور خارجه 

KeyboardInterrupt: 

In [ ]:
# Sort the dictionary based on total_count
sorted_result_map = dict(sorted(result_map.items(), key=lambda item: item[1][0], reverse=True))

# Print the sorted result
for key, value in sorted_result_map.items():
    print(f"{key}: {value}")

In [ ]:
for key, value in result_map.items():
    sentence = value[1]
    print(f"Sentence for {key}: {sentence}")

In [37]:

# Iterate through the result_map
for key, value in result_map.items():
    sentence = value[1]
    paragraph = doc.add_paragraph(sentence)

    # Set font size
    run = paragraph.runs[0]
    run.font.size = Pt(14)

    # Set alignment to Right-to-Left and right alignment
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(14)  # Add space after each paragraph

    # Set RTL direction
    paragraph_element = paragraph._p
    paragraph_properties = paragraph_element.get_or_add_pPr()
    bidi = OxmlElement('w:bidi')
    bidi.set(qn('w:val'), '1')
    paragraph_properties.append(bidi)

# Save the document
doc.save("news_doc.docx")